In [24]:
import redis
import pandas as pd
import re
from tqdm.notebook import tqdm
import threading
import time
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

In [25]:
rd = redis.Redis(host = "107.22.67.15",port = 6379,password = "cryogen",charset="utf-8", decode_responses=True)
rd.ping()

True

In [26]:
loaded_df = None
try:
    loaded_df = pd.read_csv("target_df.csv",index_col="scopus_id")
except:
    loaded_df = pd.DataFrame({"scopus_id" : pd.Series(dtype = "object"),
              "author_name" : pd.Series(dtype = "object"),
             "title" : pd.Series(dtype = "object"),
             "abstracts" : pd.Series(dtype = "object"),
             "related_field" : pd.Series(dtype = "object"),
             "auth_keywords" : pd.Series(dtype = "object"),
              "publish_day" : pd.Series(dtype = "object"),
              "publish_month" : pd.Series(dtype = "object"),
              "publish_year" : pd.Series(dtype = "object"),
              "ref_count" : pd.Series(dtype = "int64"),
              "ref_authors" : pd.Series(dtype = "object"),
              "ref_titles" : pd.Series(dtype = "object"),
              "cited_count" : pd.Series(dtype = "int64"),
             })
    loaded_df = loaded_df.set_index("scopus_id")
loaded_df

,author_name,title,abstracts,related_field,auth_keywords,publish_day,publish_month,publish_year,ref_count,ref_authors,ref_titles,cited_count
scopus_id,,,,,,,,,,,,
85162635794,Strand D.,RICKSHAW BEIJING: City People and Politics in ...,"In the 1920s, revolution, war, and imperialist...",ARTS,20th century chinese history;aggression;army;a...,1,1,2023,240,Li F.;Levenson J.;Li D.;Kerning C.;Alitto G.S....,Rural Elites in Transition: China's Cultural C...,112
85146137043,Cao L.;Pan Y.;Liang H.;Huang T.,Observer-Based Dynamic Event-Triggered Control...,This article is concerned with the dynamic eve...,COMP;ENGI;COMP;COMP;COMP;ENGI,Composite neural-network (NN) dynamic surface ...,1,5,2023,54,Li H.F.;Hu Q.L.;Xiang Z.R.;Chen K.;Sheng H.M.;...,Attitude synchronization of a group of spacecr...,132
85173378680,Spiegel G.M.,Romancing the Past The Rise of Vernacular Pros...,In a poststructuralist study of thirteenth-cen...,ARTS,NaN,1,1,2023,392,Tyvaert M.;Gerald of Wales.;Colker M.L.;Murphy...,Gesta Philippi Augusti;Philippide;Historia Reg...,118
85137398103,Arnold M.;Morgan E.;Rumgay H.;Mafra A.;Singh D...,Current and future burden of breast cancer: Gl...,Background: Breast cancer is the most commonly...,MEDI;MEDI;BIOC,Breast cancer;Global;Incidence;Mortality;Predi...,1,12,2022,49,Brinton L.A.;Miglioretti D.L.;Otero I.V.;Belle...,Global cancer statistics 2020: GLOBOCAN estima...,627
85113901401,Zhang L.S.;Jiang X.H.;Zhong Z.A.;Tian L.;Sun Q...,Carbon Nitride Supported High-Loading Fe Singl...,Singlet oxygen (1O2) is an excellent active sp...,CENG;CHEM,degradation;Fe single-atom catalyst;graphitic ...,27,9,2021,61,Zeng J.B.;Yi X.Y.;Li J.Q.;Li X.N.;Zhang Z.W.;L...,NaN,520
...,...,...,...,...,...,...,...,...,...,...,...,...
85186482236,Şenay R.İ.;Özdemir H.N.;Seki Y.;Keskin Ö.Y.;Da...,Comparison of Alkali Concentration to Enhance ...,The objective of the present research was to i...,MATE;ENGI,alkali treatment;banana fiber;Degumming;lignoc...,31,12,2023,56,Keskin OY;Rodriguez-Llamazares S;Hassanin AH.;...,New Brazilian pineapple leaf fbers for textile...,0
85114935248,Zhang D.;Yu S.;Wang X.;Huang J.;Pan W.;Zhang J...,UV illumination-enhanced ultrasensitive ammoni...,Ammonia has been used as an important marker t...,ENVI;ENVI;ENVI;ENVI;ENVI,(001)TiO2/Ti3C2Tx heterostructure;Ammonia gas ...,5,2,2022,53,Hua S.J.;Chludzinski T.;Zhou T.;Ko K.Y.;Zeng J...,Formaldehyde detection with chemical gas senso...,227
85074697156,Slack F.J.;Chinnaiyan A.M.,The Role of Non-coding RNAs in Oncology,"For decades, research into cancer biology focu...",BIOC,NaN,14,11,2019,223,Kosik K.S.;Magee R.;Pan Q.;Volinia S.;Yaylaogl...,Aberrant regulation and function of microRNAs ...,916


In [21]:
def multiThreadQuery(thread_size = 8,mykey = "scopus:"):
    cache = set([re.sub(mykey,"",k) for k in rd.keys(mykey + "*")])
    pull_keys = np.array(list(cache - set(loaded_df.index.values.astype(str))))
    my_zipper = []
    
    def pull(key):
        nonlocal my_zipper
        result = rd.hgetall(f"{mykey}{key}")
        my_zipper.append((key,result['author_name'],result['title'],result['abstracts'],result['related_field'],result['auth_keywords'],result['publish_day'],result['publish_month'],result['publish_year'],result['ref_count'],result["ref_authors"],result["ref_titles"],result['cited_count']))
        
    with tqdm(total = len(pull_keys)) as pbar:
        with ThreadPoolExecutor(max_workers=thread_size) as executor:
            future = [executor.submit(pull,key) for key in pull_keys]
            for f in as_completed(future):
                pbar.update(1)     
    col = ["scopus_id","author_name","title","abstracts","related_field","auth_keywords","publish_day","publish_month","publish_year","ref_count","ref_authors","ref_titles","cited_count"]
    arr = list(zip(*my_zipper))
    datapoint = {key : list(value) for key, value in zip(col,zip(*my_zipper))}
    added_df = pd.DataFrame(datapoint)
    if not added_df.empty:
        added_df['scopus_id'] = added_df['scopus_id'].astype(str)
        added_df = added_df.set_index('scopus_id')
        saved_df = pd.concat([loaded_df,added_df],axis = 0)
        saved_df.to_csv("target_df.csv")
    
    return 

In [22]:
multiThreadQuery(2048,"scopus:")

  0%|          | 0/19200 [00:00<?, ?it/s]

In [23]:
len(rd.keys("scopus:*"))

19200

In [9]:
rd.hkeys("visualize:85080065330")

['publish_month',
 'abstracts',
 'auth_keywords_list',
 'author_name',
 'ref_titles',
 'publish_day',
 'ref_authors',
 'correlation_keywords',
 'auth_keywords',
 'related_field',
 'publish_year',
 'ref_count',
 'extract_keywords',
 'title',
 'cited_count']